# Keras exercise

In this exercise you will be creating a Keras model by loading a data set, preprocessing input data, building a Sequential Keras model and compiling the model with a training configuration. Afterwards, you train your model on the training data and evaluate it on the test set. To finish this exercise, you will past the accuracy of your model to the Coursera grader.

This notebook is tested in IBM Watson Studio under python 3.6

## Data

For this exercise we will use the Reuters newswire dataset. This dataset consists of 11,228 newswires from the Reuters news agency. Each wire is encoded as a sequence of word indexes, just as in the IMDB data we encountered in lecture 5 of this series. Moreover, each wire is categorised into one of 46 topics, which will serve as our label. This dataset is available through the Keras API.

## Goal

We want to create a Multi-layer perceptron (MLP) using Keras which we can train to classify news items into the specified 46 topics.

## Instructions

We start by installing and importing everything we need for this exercise:

In [1]:
#!pip install tensorflow==2.2.0rc0

In [1]:
import tensorflow as tf
if not tf.__version__ == '2.2.0-rc0':
    print(tf.__version__)
    raise ValueError('please upgrade to TensorFlow 2.2.0-rc0, or restart your Kernel (Kernel->Restart & Clear Output)')

2.3.0


ValueError: please upgrade to TensorFlow 2.2.0-rc0, or restart your Kernel (Kernel->Restart & Clear Output)

IMPORTANT! => Please restart the kernel by clicking on "Kernel"->"Restart and Clear Outout" and wait until all output disapears. Then your changes are beeing picked up

As you can see, we use Keras' Sequential model with only two types of layers: Dense and Dropout. We also specify a random seed to make our results reproducible. Next, we load the Reuters data set:

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
seed = 1337
np.random.seed(seed)
from tensorflow.keras.datasets import reuters

max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2,
                                                         seed=seed)
num_classes = np.max(y_train) + 1  # 46 topics

2113536/2110848 [==============================] - 2s 1us/step


C:\Users\Usuario\anaconda3\envs\data_1\lib\site-packages\tensorflow\python\keras\datasets\reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\Usuario\anaconda3\envs\data_1\lib\site-packages\tensorflow\python\keras\datasets\reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [3]:
x_train.shape

(8982,)

In [4]:
y_train

array([ 4,  3,  3, ...,  3, 25, 11], dtype=int64)

Note that we cap the maximum number of words in a news item to 1000 by specifying the *num_words* key word. Also, 20% of the data will be test data and we ensure reproducibility by setting our random seed.

Our training features are still simply sequences of indexes and we need to further preprocess them, so that we can plug them into a *Dense* layer. For this we use a *Tokenizer* from Keras' text preprocessing module. This tokenizer will take an index sequence and map it to a vector of length *max_words=1000*. Each of the 1000 vector positions corresponds to one of the words in our newswire corpus. The output of the tokenizer has a 1 at the i-th position of the vector, if the word corresponding to i is in the description of the newswire, and 0 otherwise. Even if this word appears multiple times, we still just put a 1 into our vector, i.e. our tokenizer is binary. We use this tokenizer to transform both train and test features:

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

## 1. Exercise part: label encoding

Use to_categorical, as we did in the lectures, to transform both *y_train* and *y_test* into one-hot encoded vectors of length *num_classes*:

In [6]:
num_classes

46

In [7]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

## 2. Exercise part: model definition

Next, initialise a Keras *Sequential* model and add three layers to it:

    Layer: Add a *Dense* layer with in input_shape=(max_words,), 512 output units and "relu" activation.
    Layer: Add a *Dropout* layer with dropout rate of 50%.
    Layer: Add a *Dense* layer with num_classes output units and "softmax" activation.

In [8]:
max_words

1000

In [9]:
model = Sequential()
model.add( Dense(512, activation='relu', 
                 input_shape=(max_words,)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

## 3. Exercise part: model compilation

As the next step, we need to compile our Keras model with a training configuration. Compile your model with "categorical_crossentropy" as loss function, "adam" as optimizer and specify "accuracy" as evaluation metric. NOTE: In case you get an error regarding h5py, just restart the kernel and start from scratch

In [10]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

## 4. Exercise part: model training and evaluation

Next, define the batch_size for training as 32 and train the model for 5 epochs on *x_train* and *y_train* by using the *fit* method of your model. Then calculate the score for your trained model by running *evaluate* on *x_test* and *y_test* with the same batch size as used in *fit*.

In [11]:
batch_size = 32
epochs = 5
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
score = model.evaluate(x_test, y_test, batch_size=batch_size)

Epoch 1/5
281/281 [==============================] - 1s 4ms/step - loss: 1.3985 - accuracy: 0.6866
Epoch 2/5
281/281 [==============================] - 1s 4ms/step - loss: 0.7694 - accuracy: 0.8232
Epoch 3/5
281/281 [==============================] - 1s 5ms/step - loss: 0.5475 - accuracy: 0.8670
Epoch 4/5
281/281 [==============================] - 1s 5ms/step - loss: 0.4237 - accuracy: 0.8987
Epoch 5/5
71/71 [==============================] - 0s 2ms/step - loss: 0.8488 - accuracy: 0.8032


If you have done everything as specified, in particular set the random seed as we did above, your test accuracy should be around 80% 

In [12]:
score[1]

0.8032057285308838

Congratulations, now it's time to submit your result to the Coursera grader by executing the following cells (Programming Assingment, Week2). 

We have to install a little library in order to submit to coursera


In [22]:
#!rm -f rklib.py
#!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
import requests, json
import zipfile
import os

def submit(submitterEmail,secret,key,submission_part, all_parts, data):
        submission = {
                    "assignmentKey": key,
                    "submitterEmail":  submitterEmail,
                    "secret":  secret,
                    "parts": {}
                  }
        for part in all_parts:
            if part == submission_part:
                submission["parts"][part] = {"output": data}
            else:
                submission["parts"][part] = dict()
        response = requests.post('https://www.coursera.org/api/onDemandProgrammingScriptSubmissions.v1', data=json.dumps(submission))
        if response.status_code == 201:
            print ("Submission successful, please check on the coursera grader page for the status")
            print ("-------------------------")
            print (response.text)
            print ("-------------------------")
        else:
            print ("Something went wrong, please have a look at the reponse of the grader")
            print ("-------------------------")
            print (response.text)
            print ("-------------------------")
            
def submitAll(submitterEmail,secret,key,parts_and_data):
        submission = {
                    "assignmentKey": key,
                    "submitterEmail":  submitterEmail,
                    "secret":  secret,
                    "parts": {}
                  }
        for part, output in parts_and_data.items():
            if output is not None:
                submission["parts"][part] = {"output": output}
            else:
                submission["parts"][part] = dict()
        response = requests.post('https://www.coursera.org/api/onDemandProgrammingScriptSubmissions.v1', data=json.dumps(submission))
        if response.status_code == 201:
            print ("Submission successful, please check on the coursera grader page for the status")
            print ("-------------------------")
            print (response.text)
            print ("-------------------------")
        else:
            print ("Something went wrong, please have a look at the reponse of the grader")
            print ("-------------------------")
            print (response.text)
            print ("-------------------------")


def zipit(target, path):
    zipf = zipfile.ZipFile(target, 'w', zipfile.ZIP_DEFLATED)
    for root, dirs, files in os.walk(path):
        for file in files:
            zipf.write(os.path.join(root, file))
    zipf.close()

Please provide your email address and obtain a submission token (secret) on the grader’s submission page in coursera, then execute the cell

In [23]:

import json

key = "XbAMqtjdEeepUgo7OOVwng"
part = "HCvcp"
email = "alvarojzsanchez@gmail.com"
token = "Y1liUOBsYdBCFvSS"


submit(email, token, 'XbAMqtjdEeepUgo7OOVwng', part, [part], json.dumps(score[1]*100))

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"ozVf2","id":"tE4j0qhMEeecqgpT6QjMdA~ozVf2~quGvGMW9EeufggoLmtdySw","courseId":"tE4j0qhMEeecqgpT6QjMdA"}],"paging":{},"linked":{}}
-------------------------


In [15]:
import torch

In [16]:
m_tensor = torch.randn((5, 4, 4, 4))
m_tensor[0]

tensor([[[ 1.2897, -0.5149,  0.5909, -0.2708],
         [-0.7482, -0.3854,  1.1178,  1.1173],
         [ 1.4740, -0.2780, -0.9321, -0.1620],
         [ 1.4759, -0.2472, -1.2656,  0.1583]],

        [[ 0.1918,  1.3452,  0.0586,  0.1794],
         [-0.2940, -0.3401,  0.3099, -1.6111],
         [ 0.2929, -1.0836,  0.1151,  1.4790],
         [ 0.9437,  0.6889, -1.9084,  0.2715]],

        [[ 0.0104,  0.5297,  1.3051, -0.1261],
         [-0.8036,  0.3344, -0.3118, -0.3025],
         [ 0.6365, -1.1358, -1.4752, -0.5862],
         [ 0.3513, -0.8056, -0.1282, -0.8160]],

        [[ 0.1947, -1.2105, -0.5114, -0.2210],
         [-0.5273,  1.3969,  1.8962, -0.8271],
         [ 0.2662,  0.0140,  0.3971, -1.0356],
         [ 0.3684,  0.0542,  1.3950,  0.4543]]])

In [17]:
x = torch.randn(4, 3)
y = torch.randn(3, 4)

In [18]:
z = torch.cat([x, y], 1)

RuntimeError: Sizes of tensors must match except in dimension 1. Got 4 and 3 in dimension 0 (The offending index is 1)